# Instructions
- Run both of these cells
- Check our examples in make_graph, you want to name your log file `<algorithm>-<hyperparameter>-<value>.tar.gz`, and
  your model file should follow similarly except you must include the word `model`,
  `<algorithm>-<hyperparameter>-<value>-<model>.tar.gz`. Because you ran all defaults except for the hyperparam, you
  just need one model as shown in the example.
- Be sure to check track name in cell 1!

In [5]:
# Intra-Project Imports
import utils

# Apply sklearn intel acceleration patch.
import platform
if 'Intel' in platform.processor():
    if utils.is_package_installed('scikit-learn-intelex'):
        from sklearnex import patch_sklearn
        patch_sklearn()

# Standard Library Imports
import copy
import json
import math
import tarfile
from pathlib import Path

# 3rd Party Imports
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
import plotly.express as px
from tqdm import tqdm

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [6]:
# Extract files and calculate metrics

# Define the track name for the analysis
# track_name='2022_reinvent_champ'
track_name='2024_American_Hills_Speedway'

# Define the number of episodes per iteration
episode_per_iter = 20

# Get the waypoints for the specified track
waypoints = utils.get_track_waypoints(track_name)

# extract track boundaries from waypoints
center_line = waypoints[:,0:2] 
inner_border = waypoints[:,2:4]
outer_border = waypoints[:,4:6]

# Get absolute path to submission_dir
make_graph_dir = Path("__file__").parent.resolve() / "make_graph"

# Walk through amke_graph directory and expand all compressed folders
for entry in tqdm(make_graph_dir.iterdir(), total=4, desc='expansion of tar.gz\'s'):
    if entry.is_file() and entry.suffix == '.gz':
        # Extract all tars
        with tarfile.open(entry.absolute(), 'r:gz') as tar:
            # Extract all contents to the specified directory
            tar.extractall(path=make_graph_dir / entry.stem.strip('.tar'))

# Find and open the model metadata JSON file
with open(next(make_graph_dir.rglob('*model_metadata.json')).as_posix()) as json_file:
    model_metadata = json.load(json_file)

# Initialize data dictionary to store metrics
data = {}

# Flag to handle the first pass
initial_pass = True
entry_names = []

# Iterate through the expanded entries in the make_graph directory and calculate metrics
for entry in tqdm(make_graph_dir.iterdir(), total=4, desc='calculating metrics'):
    # Skip over model entries
    if 'model' in entry.name:
        continue
    # Process only directories related to 'ppo' or 'sac' algorithms
    if entry.is_dir() and ('ppo' in entry.name or 'sac' in entry.name):
        data[entry.name] = {}

        # Get all JSON files in the directory
        jsons = list(entry.rglob('*.json'))

        # Get all simulation trace CSV files
        sim_trace_csvs = [path.as_posix() for path in list(entry.rglob('*iteration.csv'))]

        # Path for merged simulation trace file
        merged_simtrace_path = make_graph_dir / entry.name / "merged_simtrace.csv"
        utils.merge_csv_files(merged_simtrace_path, sim_trace_csvs)

        # Read the merged simulation trace CSV into a DataFrame
        df = pd.read_csv(merged_simtrace_path)
        
        # Calculate iteration array based on the maximum episode
        iteration_arr = np.arange(math.ceil(df.episode.max() / episode_per_iter)+ 1) * episode_per_iter
        df['iteration'] = np.digitize(df.episode, iteration_arr)
        df = df.rename(columns={'X': 'x', 'Y': 'y', 'tstamp': 'timestamp'})

        # Convert continuous data to discrete buckets
        _, df = utils.continuous_to_discrete(copy.copy(model_metadata), df, num_angle_buckets=5, num_speed_buckets=4)            

        # Scale the reward values using MinMaxScaler
        min_max_scaler = MinMaxScaler()
        scaled_vals = min_max_scaler.fit_transform(df['reward'].values.reshape(df['reward'].values.shape[0], 1))
        df['reward'] = pd.DataFrame(scaled_vals.squeeze())

        # Parse episodes and actions from the DataFrame
        action_dict, episode_dict, sorted_episodes = utils.episode_parser(df)
        sorted_episodes.sort()

        # Collect metrics for each episode:
        #   - Distance to center statistics
        stats = [utils.get_distance_to_center_stats(episode_dict[episode], center_line) for episode in sorted_episodes]
        #   - Reward
        for episode in sorted_episodes:
            stats[episode]['reward'] = df[df['episode'] == episode]['reward'].sum()
        
        # Store the collected metrics in the data dictionary
        data[entry.name] = pd.DataFrame(stats)

expansion of tar.gz's: 22it [00:06,  3.41it/s]                      
calculating metrics: 22it [01:42,  4.64s/it]                      


In [10]:
# Optional: you can save "data"
# Uncomment/edit the python lines below depending on what you need

# `data`` is a dictionary of pandas dataframes. Pandas dataframes can be converted to dictionaries. 
# We will convert all the dataframes to dictionaries and then save the dictionary of dictionaries to disk as
# a .json file.
# To load the data back up, we will load the .json from disk and then convert the dictionaries back to dataframes.

# Save the dictionary to a .json file. Note: rename 'data.json' to something more meaningful
# data_to_save = {key: value.to_dict() for key, value in data.items()}
# with open(Path('saved_data') / 'data.json', 'w+') as f:
#    json.dump(data_to_save, f)

# You can also load the saved json back up
# with open(Path('saved_data') / 'data.json', 'r') as f:
#    loaded_data = json.load(f)
#    data = {key: pd.DataFrame(value) for key, value in loaded_data.items()} # here we convert the dictionaries back into DataFrames

In [3]:
# Set the smoothing factor for rolling averages
smoothing = 0.03

# Process the data for each algorithm
for algo, df in data.items():
    df.reset_index(inplace=True)
    if not 'episode' in df.columns:
        df.rename(columns={'index': 'episode'}, inplace=True)

    # Calculate the window size for rolling averages
    window_size = int(smoothing * df.shape[0])

    # Apply rolling average to the metrics
    df['smoothed_mean'] = data[algo]['mean-distance-to-center'].rolling(window=window_size, min_periods=0).mean()
    df['smoothed_median'] = data[algo]['median-distance-to-center'].rolling(window=window_size, min_periods=0).mean()
    df['smoothed_std'] = data[algo]['std-distance-to-center'].rolling(window=window_size, min_periods=0).mean()
    df['smoothed_reward'] = data[algo]['reward'].rolling(window=window_size, min_periods=0).mean()


# Combine the processed data into a single DataFrame for plotting
combined_df = pd.concat([df.assign(algorithm=algo) for algo, df in data.items()], ignore_index=False)

# Plot median values
fig = px.line(
    combined_df,
    x='episode',
    y='smoothed_median',
    color='algorithm',
    labels={'episode': 'Episode', 'smoothed_median': 'Median Value', 'algorithm': 'Algorithm'},
    title='Median Distance to Center for Each Algorithm'
)
fig.update_layout(title={'x':0.5, 'xanchor': 'center'})
fig.show()

# Plot mean values
fig = px.line(
    combined_df,
    x='episode',
    y='smoothed_mean',
    color='algorithm',
    labels={'episode': 'Episode', 'smoothed_mean': 'Mean Value'},
    title='Mean Distance to Center for Each Algorithm'
)
fig.update_layout(title={'x':0.5, 'xanchor': 'center'})
fig.show()

# Plot std values
fig = px.line(
    combined_df,
    x='episode',
    y='smoothed_std',
    color='algorithm',
    labels={'episode': 'Episode', 'smoothed_std': 'Standard Deviation Value'},
    title='Standard Deviation Distance to Center for Each Algorithm'
)
fig.update_layout(title={'x':0.5, 'xanchor': 'center'})
fig.show()

# Plot reward values
fig = px.line(
    combined_df,
    x='episode',
    y='smoothed_reward',
    color='algorithm',
    labels={'episode': 'Episode', 'smoothed_reward': 'Reward Value'},
    title='Reward for Each Algorithm'
)
fig.update_layout(title={'x':0.5, 'xanchor': 'center'})
fig.show()